# Prepare zero-shot split 
Based on the paper: Bansal, Ankan, et al. "Zero-shot object detection." Proceedings of the European Conference on Computer Vision (ECCV). 2018.

In [1]:
import json

In [2]:
import numpy as np

In [3]:
import torch

/workspace/anaconda3/envs/codet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.modeling.language_backbone.transformers import BERT

In [5]:
with open('datasets/vindr/annotations/train_instances_coco.json', 'r') as fin:
    coco_train_anno_all = json.load(fin)

In [6]:
with open('datasets/vindr/annotations/train_instances_coco.json', 'r') as fin:
    coco_train_anno_seen = json.load(fin)

In [7]:
with open('datasets/vindr/annotations/train_instances_coco.json', 'r') as fin:
    coco_train_anno_unseen = json.load(fin)

In [8]:
with open('datasets/vindr/annotations/val_instances_coco.json', 'r') as fin:
    coco_val_anno_all = json.load(fin)

In [9]:
with open('datasets/vindr/annotations/val_instances_coco.json', 'r') as fin:
    coco_val_anno_seen = json.load(fin)

In [10]:
with open('datasets/vindr/annotations/val_instances_coco.json', 'r') as fin:
    coco_val_anno_unseen = json.load(fin)

In [11]:
with open('datasets/vindr/zero-shot/vindr_seen_classes.json', 'r') as fin:
    labels_seen = json.load(fin)

In [12]:
with open('datasets/vindr/zero-shot/vindr_unseen_classes.json', 'r') as fin:
    labels_unseen = json.load(fin)

In [13]:
len(labels_seen), len(labels_unseen)

(15, 0)

In [14]:
labels_all = [item['name'] for item in coco_val_anno_all['categories']]

In [15]:
set(labels_seen) - set(labels_all)

set()

In [16]:
set(labels_unseen) - set(labels_all)

set()

In [17]:
print(coco_val_anno_all['categories'])

[{'id': 0, 'name': 'Aortic enlargement', 'supercategory': ''}, {'id': 13, 'name': 'Pulmonary fibrosis', 'supercategory': ''}, {'id': 3, 'name': 'Cardiomegaly', 'supercategory': ''}, {'id': 14, 'name': 'No finding', 'supercategory': ''}, {'id': 11, 'name': 'Pleural thickening', 'supercategory': ''}, {'id': 9, 'name': 'Other lesion', 'supercategory': ''}, {'id': 1, 'name': 'Atelectasis', 'supercategory': ''}, {'id': 6, 'name': 'Infiltration', 'supercategory': ''}, {'id': 7, 'name': 'Lung Opacity', 'supercategory': ''}, {'id': 10, 'name': 'Pleural effusion', 'supercategory': ''}, {'id': 8, 'name': 'Nodule/Mass', 'supercategory': ''}, {'id': 4, 'name': 'Consolidation', 'supercategory': ''}, {'id': 5, 'name': 'ILD', 'supercategory': ''}, {'id': 12, 'name': 'Pneumothorax', 'supercategory': ''}, {'id': 2, 'name': 'Calcification', 'supercategory': ''}]


In [18]:
class_id_to_split = {}
class_name_to_split = {}
for item in coco_val_anno_all['categories']:
    if item['name'] in labels_seen:
        class_id_to_split[item['id']] = 'seen'
        class_name_to_split[item['name']] = 'seen'
    elif item['name'] in labels_unseen:
        class_id_to_split[item['id']] = 'unseen'
        class_name_to_split[item['name']] = 'unseen'


In [19]:
class_name_to_glove = {}
with open('datasets/coco/zero-shot/glove.6B.300d.txt', 'r') as fin:
    for row in fin:
        row_tk = row.split()
        if row_tk[0] in class_name_to_split:
            class_name_to_glove[row_tk[0]] = [float(num) for num in row_tk[1:]]


In [20]:
labels_seen

['Aortic enlargement',
 'Atelectasis',
 'Calcification',
 'Cardiomegaly',
 'Consolidation',
 'ILD',
 'Infiltration',
 'Lung Opacity',
 'Nodule/Mass',
 'Other lesion',
 'Pleural effusion',
 'Pleural thickening',
 'Pneumothorax',
 'Pulmonary fibrosis',
 'No finding']

In [21]:
bert = BERT(cfg)

In [22]:
_ = bert.to('cuda')

In [23]:
class_name_to_bertemb = {}
for c in class_name_to_split:
    inputs = bert.tokenizer(c, return_tensors='pt')
    token_ids = inputs['input_ids'][0]
    token_embeddings = bert.embeddings[token_ids]
    embedding = token_embeddings.mean(dim=0)
    # print(token_embeddings)
    # print(embedding)
    # print(inputs)
    # print(token_ids)
    # if c not in bert.tokenizer.vocab:
    #     print(f'{c} not found')
    #     continue
    # cid = bert.tokenizer.vocab[c]
    # class_name_to_bertemb[c] = bert.embeddings[cid]
    class_name_to_bertemb[c] = embedding


In [24]:
class_list = list(class_name_to_split.keys())


In [25]:
encoded_class_list = bert(class_list)

/workspace/anaconda3/envs/codet/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [26]:
mask = (1 - encoded_class_list['special_tokens_mask']).to(torch.float32)

In [27]:
mask.sum(-1)

tensor([6., 4., 5., 2., 5., 3., 3., 3., 3., 6., 4., 1., 2., 5., 4.],
       device='cuda:0')

In [28]:
embeddings = (encoded_class_list['input_embeddings'] * mask[:, :, None]).sum(1) / mask.sum(1)[:, None]

In [29]:
embeddings = embeddings.cpu().numpy()

In [30]:
embeddings.shape

(15, 768)

In [31]:
class_name_to_bertemb = {}
for c, emb in zip(class_list, embeddings.tolist()):
    class_name_to_bertemb[c] = emb

In [32]:
len(class_name_to_bertemb), len(class_name_to_glove), len(class_name_to_split)

(15, 0, 15)

In [51]:
def filter_annotation(anno_dict, split_name_list):
    filtered_categories = []
    print(split_name_list)
    for item in anno_dict['categories']:
        if class_id_to_split.get(item['id']) in split_name_list:
            print(item)
            item['embedding'] = {}
            # item['embedding']['GloVE'] = class_name_to_glove[item['name']]
            item['embedding']['BertEmb'] = class_name_to_bertemb[item['name']]
            item['split'] = class_id_to_split.get(item['id'])
            filtered_categories.append(item)
    anno_dict['categories'] = filtered_categories
    # print(filtered_categories)
    
    filtered_images = []
    filtered_annotations = []
    useful_image_ids = set()
    for item in anno_dict['annotations']:
        if class_id_to_split.get(item['category_id']) in split_name_list:
            filtered_annotations.append(item)
            useful_image_ids.add(item['image_id'])
    for item in anno_dict['images']:
        
        if item['id'] in useful_image_ids:
            filtered_images.append(item)
    anno_dict['annotations'] = filtered_annotations
    anno_dict['images'] = filtered_images    

In [34]:
print(coco_train_anno_seen['categories'])

[{'id': 14, 'name': 'No finding', 'supercategory': ''}, {'id': 3, 'name': 'Cardiomegaly', 'supercategory': ''}, {'id': 0, 'name': 'Aortic enlargement', 'supercategory': ''}, {'id': 11, 'name': 'Pleural thickening', 'supercategory': ''}, {'id': 5, 'name': 'ILD', 'supercategory': ''}, {'id': 8, 'name': 'Nodule/Mass', 'supercategory': ''}, {'id': 13, 'name': 'Pulmonary fibrosis', 'supercategory': ''}, {'id': 7, 'name': 'Lung Opacity', 'supercategory': ''}, {'id': 1, 'name': 'Atelectasis', 'supercategory': ''}, {'id': 9, 'name': 'Other lesion', 'supercategory': ''}, {'id': 6, 'name': 'Infiltration', 'supercategory': ''}, {'id': 10, 'name': 'Pleural effusion', 'supercategory': ''}, {'id': 2, 'name': 'Calcification', 'supercategory': ''}, {'id': 4, 'name': 'Consolidation', 'supercategory': ''}, {'id': 12, 'name': 'Pneumothorax', 'supercategory': ''}]


In [35]:
print(class_name_to_glove.keys())

dict_keys([])


In [36]:
print(['seen'])

['seen']


In [37]:
filter_annotation(coco_train_anno_seen, ['seen'])

['seen']
{'id': 14, 'name': 'No finding', 'supercategory': ''}
{'id': 3, 'name': 'Cardiomegaly', 'supercategory': ''}
{'id': 0, 'name': 'Aortic enlargement', 'supercategory': ''}
{'id': 11, 'name': 'Pleural thickening', 'supercategory': ''}
{'id': 5, 'name': 'ILD', 'supercategory': ''}
{'id': 8, 'name': 'Nodule/Mass', 'supercategory': ''}
{'id': 13, 'name': 'Pulmonary fibrosis', 'supercategory': ''}
{'id': 7, 'name': 'Lung Opacity', 'supercategory': ''}
{'id': 1, 'name': 'Atelectasis', 'supercategory': ''}
{'id': 9, 'name': 'Other lesion', 'supercategory': ''}
{'id': 6, 'name': 'Infiltration', 'supercategory': ''}
{'id': 10, 'name': 'Pleural effusion', 'supercategory': ''}
{'id': 2, 'name': 'Calcification', 'supercategory': ''}
{'id': 4, 'name': 'Consolidation', 'supercategory': ''}
{'id': 12, 'name': 'Pneumothorax', 'supercategory': ''}


In [52]:
filter_annotation(coco_train_anno_unseen, ['unseen'])

['unseen']


In [39]:
filter_annotation(coco_train_anno_all, ['seen', 'unseen'])

['seen', 'unseen']
{'id': 14, 'name': 'No finding', 'supercategory': ''}
{'id': 3, 'name': 'Cardiomegaly', 'supercategory': ''}
{'id': 0, 'name': 'Aortic enlargement', 'supercategory': ''}
{'id': 11, 'name': 'Pleural thickening', 'supercategory': ''}
{'id': 5, 'name': 'ILD', 'supercategory': ''}
{'id': 8, 'name': 'Nodule/Mass', 'supercategory': ''}
{'id': 13, 'name': 'Pulmonary fibrosis', 'supercategory': ''}
{'id': 7, 'name': 'Lung Opacity', 'supercategory': ''}
{'id': 1, 'name': 'Atelectasis', 'supercategory': ''}
{'id': 9, 'name': 'Other lesion', 'supercategory': ''}
{'id': 6, 'name': 'Infiltration', 'supercategory': ''}
{'id': 10, 'name': 'Pleural effusion', 'supercategory': ''}
{'id': 2, 'name': 'Calcification', 'supercategory': ''}
{'id': 4, 'name': 'Consolidation', 'supercategory': ''}
{'id': 12, 'name': 'Pneumothorax', 'supercategory': ''}


In [40]:
filter_annotation(coco_val_anno_seen, ['seen'])

['seen']
{'id': 0, 'name': 'Aortic enlargement', 'supercategory': ''}
{'id': 13, 'name': 'Pulmonary fibrosis', 'supercategory': ''}
{'id': 3, 'name': 'Cardiomegaly', 'supercategory': ''}
{'id': 14, 'name': 'No finding', 'supercategory': ''}
{'id': 11, 'name': 'Pleural thickening', 'supercategory': ''}
{'id': 9, 'name': 'Other lesion', 'supercategory': ''}
{'id': 1, 'name': 'Atelectasis', 'supercategory': ''}
{'id': 6, 'name': 'Infiltration', 'supercategory': ''}
{'id': 7, 'name': 'Lung Opacity', 'supercategory': ''}
{'id': 10, 'name': 'Pleural effusion', 'supercategory': ''}
{'id': 8, 'name': 'Nodule/Mass', 'supercategory': ''}
{'id': 4, 'name': 'Consolidation', 'supercategory': ''}
{'id': 5, 'name': 'ILD', 'supercategory': ''}
{'id': 12, 'name': 'Pneumothorax', 'supercategory': ''}
{'id': 2, 'name': 'Calcification', 'supercategory': ''}


In [41]:
filter_annotation(coco_val_anno_unseen, ['unseen'])

['unseen']


In [42]:
filter_annotation(coco_val_anno_all, ['seen', 'unseen'])

['seen', 'unseen']
{'id': 0, 'name': 'Aortic enlargement', 'supercategory': ''}
{'id': 13, 'name': 'Pulmonary fibrosis', 'supercategory': ''}
{'id': 3, 'name': 'Cardiomegaly', 'supercategory': ''}
{'id': 14, 'name': 'No finding', 'supercategory': ''}
{'id': 11, 'name': 'Pleural thickening', 'supercategory': ''}
{'id': 9, 'name': 'Other lesion', 'supercategory': ''}
{'id': 1, 'name': 'Atelectasis', 'supercategory': ''}
{'id': 6, 'name': 'Infiltration', 'supercategory': ''}
{'id': 7, 'name': 'Lung Opacity', 'supercategory': ''}
{'id': 10, 'name': 'Pleural effusion', 'supercategory': ''}
{'id': 8, 'name': 'Nodule/Mass', 'supercategory': ''}
{'id': 4, 'name': 'Consolidation', 'supercategory': ''}
{'id': 5, 'name': 'ILD', 'supercategory': ''}
{'id': 12, 'name': 'Pneumothorax', 'supercategory': ''}
{'id': 2, 'name': 'Calcification', 'supercategory': ''}


In [43]:
len(coco_val_anno_seen['categories']), len(coco_val_anno_unseen['categories']), len(coco_val_anno_all['categories'])

(15, 0, 15)

In [44]:
# coco_train_anno_seen['categories']

In [45]:
with open('datasets/vindr/zero-shot/instances_train_seen_2.json', 'w') as fout:
    json.dump(coco_train_anno_seen, fout)

In [53]:
with open('datasets/vindr/zero-shot/instances_train_unseen_2.json', 'w') as fout:
    json.dump(coco_train_anno_unseen, fout)

In [47]:
with open('datasets/vindr/zero-shot/instances_train_all_2.json', 'w') as fout:
    json.dump(coco_train_anno_all, fout)

In [48]:
with open('datasets/vindr/zero-shot/instances_val_seen_2.json', 'w') as fout:
    json.dump(coco_val_anno_seen, fout)

In [49]:
with open('datasets/vindr/zero-shot/instances_val_unseen_2.json', 'w') as fout:
    json.dump(coco_val_anno_unseen, fout)

In [50]:
with open('datasets/vindr/zero-shot/instances_val_all_2.json', 'w') as fout:
    json.dump(coco_val_anno_all, fout)